In [40]:
import pandas as pd
import spacy
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import requests
import pandas as pd
import re
from datetime import datetime
import weka.core.jvm as jvm
from weka.core.dataset import Instances, Instance
from weka.filters import Filter
from weka.core.converters import Loader, Saver
import pandas as pd
from weka.core.converters import Loader
from weka.filters import Filter
from weka.core.serialization import write, read
from weka.core.jvm import start, stop
import os
import glob
import math
import numpy as np
from datetime import datetime
import pickle
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.preprocessing import StandardScaler

In [41]:
def remove_html(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    return soup.get_text()


def remove_html_tags(text):
    text = str(text)
    return re.sub(r"<[^>]+>", "", text)


def token_similarity(string1, string2):
    def tokenize(string):
        return set(string.split())

    tokens_a = tokenize(string1)
    tokens_b = tokenize(string2)
    intersection = len(tokens_a & tokens_b)
    magnitude_a = math.sqrt(len(tokens_a))
    magnitude_b = math.sqrt(len(tokens_b))
    if magnitude_a == 0 or magnitude_b == 0:
        return 0.0
    similarity = intersection / (magnitude_a * magnitude_b)
    return similarity


def count_tokens(sentence):
    sentence = remove_html(sentence)
    return len(sentence.split())


def is_codeblock(df):
    inside_pre = False
    pre_sentences = []
    for sentence in df["sentence"]:
        if "<pre>" in sentence:
            inside_pre = True
        if inside_pre:
            pre_sentences.append(1)
        else:
            pre_sentences.append(0)
        if "</pre>" in sentence:
            inside_pre = False
    df["is_codeblock"] = pre_sentences
    return df


def api_position(sentence, api_name):
    position = sentence.find(api_name)
    return position + 1 if position != -1 else 0


def starts_lower_case(sentence):
    sentence = remove_html(sentence)
    return 1 if sentence and sentence[0].islower() else 0


def code_token_count(df):
    code_token = []
    for index, row in df.iterrows():
        sentence = row["sentence"]
        is_codeblock = row["is_codeblock"]
        cleaned_sentence = remove_html(sentence)
        if is_codeblock:
            total_length = len(sentence)
            pre_start = sentence.find("<pre>")
            pre_end = sentence.find("</pre>")
            if pre_start == -1 and pre_end == -1:
                code_token.append(total_length)
            elif pre_start == -1:
                code_token.append(pre_end)
            elif pre_end == -1:
                code_token.append(total_length - pre_start)
            else:
                soup = BeautifulSoup(sentence, "html.parser")
                code_elements = soup.find_all(["pre", "code"])
                token_num = sum(len(code.get_text()) for code in code_elements)
                code_token.append(token_num)
        else:
            soup = BeautifulSoup(sentence, "html.parser")
            code_elements = soup.find_all(["pre", "code"])
            token_num = sum(len(code.get_text()) for code in code_elements)
            code_token.append(token_num)
    df["num_code_characters"] = code_token


def contains_html_tags(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    tags = [
        "code",
        "pre",
        "a",
        "strong",
        "em",
        "i",
        "b",
        "h1",
        "h2",
        "h3",
        "sup",
        "strike",
    ]
    return any(soup.find(tag) for tag in tags)


def percentage_tagged_tokens(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    total_text = soup.get_text()
    total_tokens = len(total_text.split())
    tags_to_check = [
        "code",
        "pre",
        "a",
        "strong",
        "em",
        "i",
        "b",
        "h1",
        "h2",
        "h3",
        "sup",
        "strike",
    ]
    tagged_tokens_set = set()
    for tag in tags_to_check:
        for element in soup.find_all(tag):
            tagged_tokens_set.update(element.get_text().split())
    tagged_tokens = len(tagged_tokens_set)
    if total_tokens > 0:
        return (tagged_tokens / total_tokens) * 100
    else:
        return 0.0


def calculate_age(creation_date):
    creation_time = datetime.utcfromtimestamp(creation_date)
    current_time = datetime.utcnow()
    return (current_time - creation_time).days


def generate_embedding(new_sentences):
    def create_arff_from_list(sentences, filename):
        with open(filename, "w") as f:
            f.write("@relation sentences\n\n")
            f.write("@attribute text string\n")
            f.write("@attribute label {positive, negative}\n\n")
            f.write("@data\n")
            for sentence in sentences:
                cleaned_sentence = sentence.replace("'", "\\'").replace("\n", " ")
                f.write(f"'{cleaned_sentence}',?\n")

    loaded_filter = read("string_to_word_vector.model")
    print("Model loaded successfully.")

    loader = Loader(classname="weka.core.converters.ArffLoader")

    create_arff_from_list(new_sentences, "new_sentences.arff")
    new_data = loader.load_file("new_sentences.arff")
    new_data.class_is_last()

    filter = Filter(
        classname="weka.filters.unsupervised.attribute.StringToWordVector",
        jobject=loaded_filter,
    )
    new_vectorized_data = filter.filter(new_data)

    new_vectors = []
    for i in range(new_vectorized_data.num_instances):
        instance = new_vectorized_data.get_instance(i)
        vector = [
            instance.get_value(j)
            for j in range(1, new_vectorized_data.num_attributes - 1)
        ]
        new_vectors.append(vector)

    return new_vectors


def sent_tokenize(nlp, text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences


def time_conversion(time_string):
    time_obj = datetime.strptime(time_string, "%Y-%m-%d %H:%M:%S")
    timestamp = int(time_obj.timestamp())
    return timestamp


def fetch_features(parent, child, user, answer_id, answer_body, api):
    child_info = child[child["PostId"] == answer_id]
    question_id = child_info["ParentId"].iloc[0]
    parent_info = parent[parent["PostId"] == question_id]

    child_post_owner = child_info["OwnerUserId"].iloc[0]
    parent_post_owner = parent_info["OwnerUserId"].iloc[0]

    child_owner_repu = (
        0
        if pd.isna(child_post_owner)
        else user[user["UserId"] == child_post_owner]["Reputation"].iloc[0]
    )
    parent_owner_repu = (
        0
        if pd.isna(parent_post_owner)
        else user[user["UserId"] == parent_post_owner]["Reputation"].iloc[0]
    )

    if pd.isna(child_post_owner):
        registered = False
    else:
        registered = True

    question_features = {
        "question_contains_api_element": (
            True
            if api in parent_info["Title"].iloc[0].lower()
            or api in parent_info["Body"].iloc[0].lower()
            else False
        ),
        "question_score": parent_info["Score"].iloc[0],
        "question_views": parent_info["ViewCount"].iloc[0],
        "question_answer_count": parent_info["AnswerCount"].iloc[0],
        "question_age": calculate_age(
            time_conversion(parent_info["CreationDate"].iloc[0])
        ),
        "question_user_reputation": parent_owner_repu,
    }

    if parent_info["AcceptedAnswerId"].iloc[0] == answer_id:
        acceptance = True
    else:
        acceptance = False

    answer_features = {
        "answer_score": child_info["Score"].iloc[0],
        "answer_time_difference": (
            time_conversion(child_info["CreationDate"].iloc[0])
            - time_conversion(parent_info["CreationDate"].iloc[0])
        )
        // 60,
        "answer_size": len(answer_body),
        "answer_age": calculate_age(
            time_conversion(child_info["CreationDate"].iloc[0])
        ),
        "answer_accepted": acceptance,
        "answer_user_reputation": child_owner_repu,
        "answer_user_registered": registered,
    }

    to_return = dict()
    to_return["post_id"] = answer_id
    for i in question_features:
        to_return[i] = question_features[i]

    for i in answer_features:
        to_return[i] = answer_features[i]

    return to_return

In [ ]:
child = pd.read_csv("benchmark_csv/post_detail.csv")
parent = pd.read_csv("benchmark_csv/parent.csv")
parent.head(1)

In [ ]:
user = pd.read_csv("benchmark_csv/user.csv")
user.head()

In [ ]:
# Find some posts
df = pd.read_csv("benchmark_csv/retrieved_post.csv")
nlp = spacy.load("en_core_web_sm")

if {"api", "knowledge_type"}.issubset(df.columns):
    second_to_fifth_rows = df
    print(second_to_fifth_rows)
else:
    print("The columns 'api' and/or 'knowledge_type' do not exist in the dataframe.")

In [45]:
sentence_data = []

for index, row in second_to_fifth_rows.iterrows():
    api = row["api"]
    post = row["post"]
    post_id = row["post_id"]
    doc = nlp(post)
    sentences = [sent.text for sent in doc.sents]
    for sentence in sentences:
        sentence_data.append(
            {"api": api, "post": post, "post_id": post_id, "sentence": sentence}
        )

sentence_df = pd.DataFrame(sentence_data)

sentence_df.to_csv("post_sentence_split.csv", index=False)
print("Sentences have been split and saved to 'post_sentence_split.csv'.")

Sentences have been split and saved to 'post_sentence_split.csv'.


In [ ]:
df = sentence_df
df["num_tokens"] = df["sentence"].apply(count_tokens)
df = is_codeblock(df)
df["sentence_position"] = df.groupby("post_id").cumcount() + 1
df["api_position"] = df.apply(
    lambda row: api_position(row["sentence"], row["api"]), axis=1
)
df["starts_lower_case"] = df["sentence"].apply(starts_lower_case)
df["num_code_characters"] = code_token_count(df)
df["contains_html_tags"] = df["sentence"].apply(contains_html_tags)
df["percentage_tagged_tokens"] = df["sentence"].apply(percentage_tagged_tokens)

parent = pd.read_csv("benchmark_csv/parent.csv")
child = pd.read_csv("benchmark_csv/post_detail.csv")
user = pd.read_csv("benchmark_csv/user.csv")

all_features = []
for index, row in df.iterrows():
    post_id = row["post_id"]
    answer_body = row["post"]
    api = row["api"]
    features = fetch_features(parent, child, user, post_id, answer_body, api)
    all_features.append(features)

for i in features:
    temp = []
    for j in all_features:
        temp.append(j[i])
    df[i] = temp

In [ ]:
df["sentence"] = df["sentence"].apply(remove_html_tags)

df = df.fillna(0)
for col in df.select_dtypes(include=["bool"]).columns:
    df[col] = df[col].astype(int)

In [ ]:
start()
test_sentences = df["sentence"].tolist()
test_embeddings = generate_embedding(test_sentences)
test_embeddings_df = pd.DataFrame(test_embeddings, index=df.index)
df = pd.concat(
    [df.reset_index(drop=True), test_embeddings_df.reset_index(drop=True)], axis=1
)

In [51]:
folder_path = "official_documents"

txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

official_document_dict = {}

for file_path in txt_files:
    api = file_path.split("\\")[-1][:-4]
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
        official_document_dict[api] = sent_tokenize(nlp, content)

In [52]:
maxes = []
averages = []

for index, row in df.iterrows():
    api = row["api"]
    sentence = row["sentence"]
    sents = official_document_dict[api.lower()]
    similarities = []
    for s in sents:
        similarities.append(token_similarity(s, sentence))
    maxes.append(max(similarities))
    averages.append(np.mean(similarities))

df["max"] = maxes
df["mean"] = averages

In [53]:
df.to_csv("data_to_be_predicted_by_sise.csv", index=False)

In [54]:
with open("svm_model.pkl", "rb") as f:
    svm = pickle.load(f)
print("SVM model loaded successfully.")

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
print("scaler model loaded successfully.")

SVM model loaded successfully.
scaler model loaded successfully.


In [ ]:
info_columns = ["api", "post", "post_id", "sentence"]
feature_columns = [col for col in df.columns if col not in info_columns]

X_test = df[feature_columns]
X_test.columns = X_test.columns.astype(str)
X_test = X_test[scaler.feature_names_in_]
X_test = scaler.transform(X_test)

y_pred = svm.predict(X_test)

In [56]:
df["prediction"] = y_pred

predicted_ones = df[df["prediction"] == 1]

In [58]:
output_folder = "sise_output"
os.makedirs(output_folder, exist_ok=True)

for api, group in predicted_ones.groupby("api"):
    file_path = os.path.join(output_folder, f"{api}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\nlonglivebonankou\n".join(list(set(group["sentence"]))))
        print(f"Sentences for API '{api}' written to {file_path}.")

Sentences for API 'AccessibilityRequestPreparer' written to sise_output\AccessibilityRequestPreparer.txt.
Sentences for API 'ActionBar' written to sise_output\ActionBar.txt.
Sentences for API 'AdapterStateHelper' written to sise_output\AdapterStateHelper.txt.
Sentences for API 'AlgorithmConstraints' written to sise_output\AlgorithmConstraints.txt.
Sentences for API 'Attr' written to sise_output\Attr.txt.
Sentences for API 'Attributes2Impl' written to sise_output\Attributes2Impl.txt.
Sentences for API 'BigDecimal' written to sise_output\BigDecimal.txt.
Sentences for API 'ByteArray' written to sise_output\ByteArray.txt.
Sentences for API 'CharSequence' written to sise_output\CharSequence.txt.
Sentences for API 'CheckBoxPreference' written to sise_output\CheckBoxPreference.txt.
Sentences for API 'CheckpointManager' written to sise_output\CheckpointManager.txt.
Sentences for API 'ConnectionEvent' written to sise_output\ConnectionEvent.txt.
Sentences for API 'DatasetSpec' written to sise_ou